In [ ]:
# !pip install transformers==4.43.2
!pip install bitsandbytes==0.43.3
!pip install -U bitsandbytes
!pip install accelerate==0.33.0

In [ ]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

HF_TOKEN = os.getenv("TOKEN_NAME")
login(token=HF_TOKEN)

In [ ]:
def load_model(base_model, torch_dtype=torch.bfloat16, use_quantization=False, quantization_config=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    attn_implementation = "eager"

    if use_quantization:
        if quantization_config is None:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch_dtype,
                bnb_4bit_use_double_quant=True,
            )
    else:
        bnb_config = None

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch_dtype,
        device_map="auto",
        attn_implementation=attn_implementation
    )

    if not use_quantization:
        model.to(device)

    return model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
model = load_model(model_name, use_quantization=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = model.config


print(f"Memory Allocated: {round(torch.cuda.memory_allocated() / (1024 ** 2), 3)}GB")
print(f"Memory Reserved: {round(torch.cuda.memory_reserved() / (1024 ** 2), 3)}GB")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 프롬프트 작성하기

In [ ]:
example = """
'''
이 제품은 가격대비 가성비가 좋고, 토핑도 풍부해서 맛있다는 소비자들의 평이 많아요. 집에서 간편하게 끓여 먹을 수 있어서 편리하고, 양도 푸짐해서 가족 모두에게 적합해요. 다만, 유통기한이 짧아서 버리는 경우가 있었다는 의견도 있어요.
'''

'''
가격 대비 만족스러운 제품이에요. 순두부찌개나 차돌박이와 함께 먹으면 더 맛있어요. 밀키트의 장점을 모두 갖춘 제품으로 편리해요. 고기가 푸짐하고 씹는 재미도 있어서 두 명이 함께 먹기 좋아요. 신선한 재료로 만들어져 있어 매번 자주 구매하는 제품입니다. 백반집 수준의 퀄리티에 매우 만족하실 거예요.
'''

'''
가격 대비로 만족하고, 집에서 간편하게 전문점 맛을 느낄 수 있어요. 청국장이 특히 좋다는 의견도 많아요. 양도 푸짐하고, 냄새도 거의 없어서 신선한 재료를 사용한 것 같아요.
'''

'''
신선하고 좋은 제품이라고 하셔서 만족스러우시다면, 가성비가 좋아서 자주 구매하시는 고객님들이 많으실 것 같아요. 집에 추가 재료를 넣어 맛을 더해도 좋다는 팁도 있어요. 대학생 아이들이 혼자 먹기에도 편리하다는 이야기도 있네요. 스팸과 김치를 함께 끓여먹으면 더 맛있다는 소중한 정보도 함께 전해드릴게요!
'''

'''
요리하기 편하고 빠르게 먹을 수 있는 제품이에요. 냄비를 바로 끓일 수 있어서 편리하고, 12인용으로 적당한 양이에요. 혼자 먹기에도 적당하고, 맛있어서 가게에서 사먹는 것 같아요. 맵지 않아서 부드럽고, 부재료를 더 넣어서 더 맛있게 먹을 수 있어요. 만족스러운 제품이라 재구매 의사가 있습니다!
'''
"""

In [ ]:
example_recipe = """
'''
이렇게 활용하면 좋아요 : 만둣국, 갈비탕
추가하면 좋아요 : 물, 대파, 기름 적은 소고기, 밥
조리 방법 : 물을 먼저 넣고 팔팔 끓이다가 소고기를 넣어주세요. 마지막에 밥을 말아 드세요.
더 좋은 이유 : 냄비에 한 번에 넣고 끓이면 끝
'''

'''
이렇게 활용하면 좋아요 : 해물 오뎅탕
추가하면 좋아요 : 해산물, 오뎅 사리, 무, 대파, 후추, 소금
'''

'''
추가하면 좋아요 : 샤브샤브소고기, 차돌박이, 칼국수면
조리 방법 : 다 먹고 칼국수면을 넣어서 끓여 드세요.
더 좋은 이유 : 전자레인지에 돌리기만 해도 완성
'''

'''
이렇게 활용하면 좋아요 : 김치찌개, 라면
추가하면 좋아요 : 김치, 라면사리, 각종 야채
조리 방법 : 김치를 많이 넣고, 각종 야채는 조금만 추가해주세요.
'''

"""

### 데이터 적용하기(문장별)

In [ ]:
import pandas as pd

# Set pandas options to display full content of DataFrame
pd.set_option('display.max_colwidth', None)  # No truncation of column width
pd.set_option('display.max_rows', None)      # No truncation of rows
pd.set_option('display.max_columns', None)   # No truncation of columns

origin2 = pd.read_csv("/content/drive/MyDrive/kurly/카테고리_태깅된_문장_최종.csv")
origin2.head()

,product_id,product_type,review,category
0,5013198,한우 도가니탕,하트찜꽁 쫀득한 도가니탕 맛있네요,맛/향/풍미
1,5013198,한우 도가니탕,국 간은 싱거운데 소금 간 하면 딱이에요,맛/향/풍미
2,5013198,한우 도가니탕,가격은 음 착하지는 않습니다,가격
3,5013198,한우 도가니탕,기름 너무 많아서 느끼함,맛/향/풍미
4,5013198,한우 도가니탕,믿고 먹는 설성 목장이예요,맛/향/풍미


In [ ]:
def extract_assistant_text(outputs):
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()

In [ ]:
test_product_id = 5035901  #test data

filtered_data = origin2[origin2['product_id'] == test_product_id]

grouped_data2 = filtered_data.groupby(['product_type', 'category'])['review'].apply(lambda x: ' '.join(x.drop_duplicates())).reset_index()

grouped_recipe = grouped_data2[grouped_data2['category'] == '활용 방법']

grouped_data2 = grouped_data2[grouped_data2['category'] != '활용 방법']
grouped_data2 = grouped_data2[grouped_data2['review'].apply(lambda x: len(x.split()) >= 10)]

print("Grouped Recipe DataFrame (Grouped by product_type):")
display(grouped_recipe)

print("Filtered Grouped DataFrame (excluding '활용 방법', Grouped by product_type):")
display(grouped_data2)

Grouped Recipe DataFrame (Grouped by product_type):


,product_type,category,review
5,돼지 김치찌개,활용 방법,해동 후 끓여 주기만하면 되는 음식이라 조리도 굉장히 편해요 집에 양파 돼지고기 등 다른 재료 있으면 같이 넣고 끓이면 더 푸짐하고 맛있는 찌개가 될 거예요 김치 띠 개는 이것만 시켜머거요 나는 야 요리왕 김치찌개 국물이 필요하신 분이라면 오모리 김치찌개를 시켰는데 김치도 참치도 넣어야 하면 다른 거 시키세요 만두 넣어 먹었어여 돼지고기랑 대파만 추가하면 맛난 찌개 완성 됩니다 요리 귀찮은 날 간단하게 먹기 좋을 것 같아요 고기 야채 추가해서 먹었는데 달걀말이랑 싹싹 비벼서 먹으니 맛 좋습니다
10,참치 김치찌개,활용 방법,급히 김치찌개 땡길 때 참치 캔 하나 따 넣으면 괜찮은 듯합니다 햄이랑 두부 파 더 추가해서 먹어요 두부만 더 넣어서 끓여 먹으면 되요 참치로 샀는데요 저는 여기다 참치 한 캔 더 넣고 묵어요 두부 넣고 김치찌개로 먹기도 좋고 콩비지 한 팩 사 넣으면 맛있는 비지찌개가 돼요 간편해요 두부랑 참치 캔 하나 더 추가해서 끓여 먹어요 두부 추가했어요 참치 두부 추가해서 아주 맛있게 먹었네요


Filtered Grouped DataFrame (excluding '활용 방법', Grouped by product_type):


,product_type,category,review
0,돼지 김치찌개,가격,가성비 있고 맛있는 식사였어요 세일까지 하면 완전 가성비 김치찌개 대 추천 합니다
1,돼지 김치찌개,맛/향/풍미,국물이 맛있으니 면도 당연히 맛있었구요 제가 끓인 줄 알고 제가 한 요리 중에 이 김치찌개가 젤 맛 잇다는데 아주 잘 속고 있어요 김치 국물로 만들었다는 오해를 사기에 충분합니다 정말 해도 너무하네요 김치는 너무 맛있는 데 돼지고기 냄새가 너무 많이 나요 이게 무슨 김치찌게인가요 그냥 김치 국임 맛있다고 한 사람들이 해가 안 가 내요 좀 많이 새콤하긴했는데 맛있게 먹었어요 먹어 본 시판 김치찌개 중 제일 맛있습니다 맛있어요 고기가 종종 냄새가 납니다 돼지 냄새 너무 나요 김치도 흐믈거리고 국물 시원하고 맛있어요 맛은 다 아실 듯 오모리 시콤하니 맛 좋습니자 김치찌개는 오모가리가 진짜 맛있는 것 같아요 냉동 제품 치고 훌륭한 오모리 김치찌개 퀄리티에요 100 프로 시원해서 또 생각날 고 같아요
3,돼지 김치찌개,양,김치도 푹 익어서 부드럽고 고기가 정말 많이 들어 있었어요 간혹 김치나 고기가 적은 경우는 봤어도 건더기가 정말 1도 없는 얼린 국물만 들어 있네요 내용물에 김치는 하나도 없고 고기만 몇 점 있네요 양도 2 인이 먹기 적당해요 김치찌개 주문했는데 돼지고기 몇 점 김치 조금 넣어서 배송했네요 너무 부실해서 실망하고 고기랑 김치도 많이 들어 있고 충분했어요
4,돼지 김치찌개,포장/배송,부디 다음부터는 정상적인 제품으로 보내 주시길 후기를 보니 제가 주문한 김치찌개에서 빠진 김치가 다른 분들에게 갔나 봐요
7,참치 김치찌개,맛/향/풍미,고기에서 누린내가 나요 맛있어서 자주 시켜 먹습니다 쫌 짜지만 국물 맛은 생각보다 좋았어요 참치 김치찌개가 맛있는 것 같아서 맛이 깔끔하고 꽤 맛있어요 김치찌개 제품 중에 제일 맛있어요 두부 추가해서 먹으니 맛있어요 살짝 감질 맛 나그등요 아모튼 맛이 매우 괜 춘합니다 여기 김치찌개가 저의 최애입니다 맛있아 요 좋아요 맛있네요 돼지고기는 냄새가 나서 돼지고기 냄새 나요 냄새만 않나면 좋겠어요 돼지 찌개 맛있어서 오모가리 김치찌개 진짜 맛있어요 비슷한 패키지의 타 브랜드 완제품들이 더 나아유 맛은 그냥 김치찌개 맛 이거 맛있음 가장 맛있는 내가 알고 있던 기본 맛입니다
9,참치 김치찌개,양,참치는 거의 없다고 보시면 될 듯 대용량도 나오면 좋겠어요 맛있는 데 양이 김밥천국의 반 느낌 건더기도 부실해요


In [ ]:
summary_data = []

category_instructions = {
    '가격': "가격에 대한 구체적인 장점과 단점을 간결하게 요약해 주세요. 가격 대비 품질, 비슷한 제품과의 비교 등을 포함하세요.",
    '맛/향/풍미': "맛과 향에 대한 장점과 단점을 간결하고 명확하게 요약해 주세요. 맛의 특징과 문제점을 자연스럽게 설명하세요.",
    '양': "제공된 양에 대한 장단점을 간결한 문장으로 요약해 주세요. 양에 대한 만족도나 불만을 구체적으로 설명하세요.",
    '포장/배송': "포장 상태와 배송 과정에 대한 고객의 경험을 간결하게 요약해 주세요. 문제가 있었던 경우 그 이유도 설명해 주세요.",
    '보관': "제품의 보관 방법에 대한 고객의 의견을 요약해 주세요. 보관 방법의 장단점과 주의 사항을 간단하게 설명해 주세요."
}

for _, row in grouped_data2.iterrows():
    product_type = row['product_type']
    category = row['category']
    combined_reviews = row['review']

    sentiment_instructions = category_instructions.get(category, "일반적인 내용을 중심으로 요약해 주세요.")

    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 간결하고 자연스럽게 요약하는 전문가입니다.

    **지시사항:**
    리뷰: {combined_reviews}

    - 리뷰를 읽고, 각 리뷰의 핵심 내용만 간결하게 요약하세요.
    - 리뷰를 읽고, 문맥을 고려하여 요약하세요.
    - 사용자의 실제 경험을 반영하여 자연스러운 문장으로 작성하세요.
    - 동일한 내용이 반복되지 않도록 중복된 정보는 하나의 문장으로 통합하세요.
    - 각 카테고리와 관련된 정보만 포함하고, 필요 없는 정보는 생략하세요.
    - 읽는 사람이 바로 이해할 수 있도록 단순하고 직관적인 문장을 사용하세요.
    - '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.

    **카테고리별 요약 지침:**
    - {category_instructions.get(category)}

    **제한사항:**
    - 브랜드 이름은 언급하지 마세요.
    - '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.
    - 요약을 할 때 복잡한 문장보다는 자연스럽고 간결한 문장을 사용하세요.
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id,
                             repetition_penalty=1.0,
                             top_p=0.90,
                             temperature=0.2
                             )

    initial_summary = extract_assistant_text(outputs)

    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 요약 전문가입니다.

    **지시사항:**
    initial summary: {initial_summary}
    위 내용을 명확하고 간결하게 1-2문장으로 요약해 주세요.
    '소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.
    일관된 문장 형식을 유지해주세요.
    존댓말로 자연스럽게 작성해 주세요.

    **요약:**
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)

    summary_data.append([test_product_id, product_type, category, final_summary])

summary_df = pd.DataFrame(summary_data, columns=['product_id', 'product_type', 'category', 'summary'])

,product_id,product_type,category,summary
0,5035901,돼지 김치찌개,가격,김치찌개가 가성비 있고 맛있었습니다. 세일 때는 더욱 저렴한 가격에 맛있는 식사를 즐길 수 있습니다.
1,5035901,돼지 김치찌개,맛/향/풍미,"이 제품은 맛있는 국물과 면을 제공하며, 김치 국물로 만든 줄 알고 있었는데, 강한 김치와 돼지고기 냄새로 놀랐습니다. 맛있는 김치와 시원한 국물이 특징입니다."
2,5035901,돼지 김치찌개,양,"양의 양이 적은 경우가 있지만, 대부분의 경우 양이 적당합니다."
3,5035901,돼지 김치찌개,포장/배송,제품에 포함된 김치가 빠져서 다른 분들에게 보내졌습니다.
4,5035901,참치 김치찌개,맛/향/풍미,"이 제품은 고기에서 나는 맛이 좋으며, 국물 맛도 생각보다 좋습니다. 특히 두부를 추가하면 더욱 맛있답니다."
5,5035901,참치 김치찌개,양,식재가 부족하고 맛이 떨어집니다.
